<a href="https://colab.research.google.com/github/savindumahasen/AI-Detection-System-/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install the necessary libraries
!pip install pandas
!pip install numpy

In [12]:
# import necessary libraries
import pandas as pd
import numpy as np
import csv

Skipping, found downloaded files in "./ai-vs-human-text" (use force=True to force download)


In [13]:
dataset = pd.read_excel("/content/Dataset.xlsx")

dataset.head()

,sr.no,text,source,label_name,label_id
0,0,"Produced by Chris Young Ritzen, Tamara Weitzma...",openwebtext,human-written,1
1,1,"Lawrence Eugene Doby (December 13, 1923 – June...",openwebtext,human-written,1
2,2,Shock and Awe\n\nGabe Newell is about to make ...,openwebtext,human-written,1
3,3,The Best Answers to Tough Interview Questions\...,openwebtext,human-written,1
4,4,Contents:\n\n1.) Preamble\n\n2.) Understand Yo...,openwebtext,human-written,1


In [14]:
## get the information  of dataset
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11580 entries, 0 to 11579
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sr.no       11580 non-null  int64 
 1   text        11580 non-null  object
 2   source      11580 non-null  object
 3   label_name  11580 non-null  object
 4   label_id    11580 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 452.5+ KB


In [15]:
## length of the dataset
len(dataset)

11580

In [16]:
# get the value counts of label_id
dataset['label_id'].value_counts()

,count
label_id,
1,5790
0,5790


In [17]:
## split the dataset into  train and test set
from sklearn.model_selection  import train_test_split
train,test=train_test_split(dataset,test_size=0.1, random_state=42, shuffle=False)
train.shape,test.shape

((10422, 5), (1158, 5))

In [18]:
train.head()

,sr.no,text,source,label_name,label_id
0,0,"Produced by Chris Young Ritzen, Tamara Weitzma...",openwebtext,human-written,1
1,1,"Lawrence Eugene Doby (December 13, 1923 – June...",openwebtext,human-written,1
2,2,Shock and Awe\n\nGabe Newell is about to make ...,openwebtext,human-written,1
3,3,The Best Answers to Tough Interview Questions\...,openwebtext,human-written,1
4,4,Contents:\n\n1.) Preamble\n\n2.) Understand Yo...,openwebtext,human-written,1


In [19]:
test.head()

,sr.no,text,source,label_name,label_id
10422,10422,I have heard a lot about the abuse thing about...,paraphrase,ai-generated,0
10423,10423,It really pisses me off when humans are calle...,paraphrase,ai-generated,0
10424,10424,"Recently, I often see the uproar of people str...",paraphrase,ai-generated,0
10425,10425,"Before the negative voices are showered at me,...",paraphrase,ai-generated,0
10426,10426,If I don't talk about it in public or with my ...,paraphrase,ai-generated,0


## Data Preprocessing

In [ ]:
train.shape

(10422, 5)

In [ ]:
## check  the duplicate values of train set
train.duplicated().sum()

np.int64(0)

In [ ]:
## check the null value of train set
train.isnull().sum()

,0
sr.no,0
text,0
source,0
label_name,0
label_id,0


In [ ]:
train['label_id'].head()

,label_id
0,1
1,1
2,1
3,1
4,1


In [11]:
## seperate the target variable from indepdentdant variables
train_set=train.drop(['label_id','label_name','source','sr.no'],axis=True)
train_set.head()

,text
0,"Produced by Chris Young Ritzen, Tamara Weitzma..."
1,"Lawrence Eugene Doby (December 13, 1923 – June..."
2,Shock and Awe\n\nGabe Newell is about to make ...
3,The Best Answers to Tough Interview Questions\...
4,Contents:\n\n1.) Preamble\n\n2.) Understand Yo...


### Text preprocessing

In [22]:
## import text preprocessing libraries
import re
import string

In [23]:
## convert uppercase letters into  lowercase letters

In [24]:
train_set['text']=train_set['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [25]:
train_set['text'].head()

,text
0,"produced by chris young ritzen, tamara weitzma..."
1,"lawrence eugene doby (december 13, 1923 – june..."
2,shock and awe gabe newell is about to make a p...
3,the best answers to tough interview questions ...
4,contents: 1.) preamble 2.) understand your soc...


In [26]:
## remove the links from text

In [28]:
train_set['text']=train_set['text'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*','', x,flags=re.MULTILINE) for x in x.split()))
train_set['text'].head()

,text
0,"produced by chris young ritzen, tamara weitzma..."
1,"lawrence eugene doby (december 13, 1923 – june..."
2,shock and awe gabe newell is about to make a p...
3,the best answers to tough interview questions ...
4,contents: 1.) preamble 2.) understand your soc...


In [29]:
## remove punctuations

In [31]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [32]:
def remove_punctuation(text):
  for punctuations in string.punctuation:
     text=text.replace(punctuations,'')
  return text

In [34]:
train_set['text']= train_set['text'].apply(remove_punctuation)
train_set['text'].head()

,text
0,produced by chris young ritzen tamara weitzman...
1,lawrence eugene doby december 13 1923 – june 1...
2,shock and awe gabe newell is about to make a p...
3,the best answers to tough interview questions ...
4,contents 1 preamble 2 understand your social s...


In [35]:
## remove the numbers from text

In [37]:
train_set["text"]=train_set["text"].str.replace('/d+','', regex=True)

In [38]:
train_set['text'].head()

,text
0,produced by chris young ritzen tamara weitzman...
1,lawrence eugene doby december – june was a...
2,shock and awe gabe newell is about to make a p...
3,the best answers to tough interview questions ...
4,contents preamble understand your social sur...


In [39]:
## remove  stopwords

In [40]:
import nltk

In [41]:
nltk.download('stopwords', download_dir="words/")

[nltk_data] Downloading package stopwords to words/...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [42]:
with open ('words/corpora/stopwords/english', 'r') as file:
    sw=file.read().splitlines()

In [43]:
sw

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [44]:
train_set["text"]= train_set["text"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))

In [45]:
train_set['text'].head()

,text
0,produced chris young ritzen tamara weitzman pa...
1,lawrence eugene doby december – june american ...
2,shock awe gabe newell make promise overcast mo...
3,best answers tough interview questions tell re...
4,contents preamble understand social surroundin...


In [46]:
## remove the inflectional form from text

In [47]:
from nltk.stem import PorterStemmer
ps=PorterStemmer()

In [48]:
train_set["text"]=train_set["text"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))

In [49]:
train_set['text'].head()

,text
0,produc chri young ritzen tamara weitzman patti...
1,lawrenc eugen dobi decemb – june american prof...
2,shock awe gabe newel make promis overcast morn...
3,best answer tough interview question tell real...
4,content preambl understand social surround cla...


In [60]:
train_set['text'].tail()

,text
10417,im big star war fan cinema eu purpos discus wo...
10418,edit ive provid delta thread answer provid del...
10419,believ abbott govern polici attempt polici who...
10420,inspir articl outrag someon build someth home ...
10421,littl background year old femal devot first tw...


In [59]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
lemmertizer=WordNetLemmatizer()

# Download required data once
nltk.download('wordnet')
nltk.download('omw-1.4')

train_set["text"]=train_set["text"].apply(lambda x: " ".join(lemmertizer.lemmatize(x, pos='v') for x in x.split()))
train_set['text'].tail()


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,text
10417,im big star war fan cinema eu purpos discus wo...
10418,edit ive provid delta thread answer provid del...
10419,believ abbott govern polici attempt polici who...
10420,inspir articl outrag someon build someth home ...
10421,littl background year old femal devot first tw...


### Building the vocabulary

In [64]:
from collections import Counter
vocab=Counter()

In [65]:
for sentence in train_set['text']:
   vocab.update(sentence.split())

In [66]:
len(vocab)

109606

In [67]:
vocab

Counter({'produc': 2348,
         'chri': 395,
         'young': 2271,
         'ritzen': 1,
         'tamara': 1,
         'weitzman': 1,
         'patti': 34,
         'aronofski': 2,
         'nov': 83,
         'juri': 287,
         'seminol': 3,
         'counti': 643,
         'florida': 392,
         'take': 13457,
         'less': 4319,
         'three': 3201,
         'hour': 2343,
         'convict': 467,
         'anita': 174,
         'smithey': 65,
         'seconddegre': 13,
         'murder': 950,
         'admit': 727,
         'shoot': 1998,
         'kill': 2638,
         'estrang': 13,
         'husband': 449,
         'robert': 551,
         'cline': 31,
         'say': 22289,
         'rape': 922,
         'think': 15056,
         'go': 15060,
         'tell': 5300,
         'truth': 748,
         'terribl': 483,
         'injustic': 130,
         'do': 2565,
         'lay': 477,
         'fact': 3988,
         'juror': 110,
         'get': 19628,
         'right':

In [74]:
def save_vocabulary(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w', encoding="utf-8")
    file.write(data)
    file.close()

save_vocabulary(vocab, 'vocabulary/vocabulary.txt')